# 의약학 : 설명을 위한 로지스틱 회귀분석

1) 종속변수 y : 심근경색 질환 (1:유 0: 무)

2) 독립변수 x : 9개
 * age: 나이 ,연속형
 * sex: 성별 (1:남, 0: 여) , 범주형
 * trestbps : 혈압 (mmHg로 측정, 94에서 200까지의 연속 값), 연속형
 * chol :  혈청 콜레스테롤 (mg / dl 단위), 연속형
 * thalach : 최대심박수
 * HbA1c : 당화 혈당, 연속형 (%)
 * fbg : 공복혈당, 연속형 (mg/dL)
 * smoke : 흡연 경험 (1:유, 0:무)
 * BMI : 체질량 지수, 연속형 (<18.5: 저체중,18.5< <23: 정상, 23< <25: 과체중, 25< <30: 과체중, 30< <35 :비만, >35 :고도비만)
 
 
 * 돌연사의 80%를 차지하고 발병 후 초기 사망률도 30%에 달하는 질병이 심근경색이다. 심근경색은 심장에 혈액을 공급하는 관상동맥에 혈전이 막아 심장근육으로 혈액을 공급하지 못하여 발생하는 질병이다. 높은 사망률을 심근경색의 원인은 음주, 흡연, 스트레스, 만성질환 등 여러 원인이 있으나, 예측하기가 어렵고 발병 즉시 빠른 처치가 가장 우선이다. 심근경색이 발 생하면 병원 도착 전 1/3이 사망에 이르게 되는 무서운 질병이다. 최근 사회적 현상으로 급성심근경색이 과거에 비해 증가하고 있는 추세에 있다
 * 심근경색은 50대 이상, 남성, 고혈압, 고지혈증, 비만, 흡연 경험등이 주요 위험인자로 알려져 있음 
    https://www.dailymedi.com/detail.php?number=823953&thread=14r02

###  데이터 불러오기


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# 샘플 데이터 불러오기 
df=pd.read_csv("heartdisease.csv")
df

In [ ]:
df.describe()

#### 종속변수와 독립변수의 관계

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
def x_hist_by_y (variable):
    sns.distplot(df[df.target == 1][variable], color="blue", label="1")
    sns.distplot(df[df.target == 0][variable], color="green", label="0")
    plt.legend(title="target")
    plt.show()

In [ ]:
X=df.drop(["target"],axis=1)
y=df["target"]

In [ ]:
# 심근경색 질환 유무에 따른 X 변수의 분포 : 나이, 흡연력, BMI의 분포가 심근경색 유무에 따라 큰 차이를 보임 
for i in X.columns :
    x_hist_by_y (i)

#### 다중공선성 분석

In [ ]:
df1 = X.corr() # correlation
mask = np.zeros_like(df1, dtype=np.bool)
mask[np.triu_indices_from(mask)]= True
f, ax = plt.subplots(figsize=(11, 9))
ax = sns.heatmap(df1, cmap = 'coolwarm', square = True, mask = mask,
                 vmin = -0.4, vmax = 0.4, annot = True, annot_kws = {"size": 10})

#### 다중공선성 분석
* fbs, HbA1c는 상관계수 0.88로 높은 상관계수를 지님 

### 로지스틱 회귀분석 (Logistic regression)

#### Model 1 : 기본 모델

In [ ]:
import statsmodels.api as sm

model_full = sm.Logit(y, sm.add_constant(X))
result_full = model_full.fit()
result_full.summary()

#### Model 2: 변수 선택후 모델링

In [ ]:
X_reduced1= X.drop(["thalach","HbA1c"],axis=1)
model_reduced1 = sm.Logit(y, sm.add_constant(X_reduced1))
result_reduced1 = model_reduced1.fit()
result_reduced1.summary()

#### 변수별 위험도

In [ ]:
np.exp(result_reduced1.params)

# 머신러닝 방법 비교
## 데이터 구분 : 학습 데이터와 검증데이터
* 전체 데이터를 학습 데이터와 검증 데이터로 50:50으로  구분
* random seed = 1234 / stratify=y (학습과 검증 데이터에서 event 발생 비중을 유지하도록 구분)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
def data_split (x, y) : 
    X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.5, random_state=1234, stratify=y)
    print("Ratio of the event: Training dataset-",round( sum(Y_train)/len(Y_train)*100,2),"%, Test dataset -" ,round( sum(Y_test)/len(Y_test)*100,2), "%") 
    return X_train, X_test, Y_train, Y_test

In [ ]:
X_train, X_test, y_train, y_test=data_split (X,y)

##  Model1 :  로지스틱 회귀모델

* 변수선택 과정을 통해 유의수준 5% 하에서 유의하지 않았던 thalach 최대 심박수 변수와 HbA1c 당화혈당 변수를 제외한 모델

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# 변수 선택후 X 변수
X_train_reduced =  X_train.drop(['thalach','HbA1c'],axis = 1)
X_test_reduced = X_test.drop(['thalach','HbA1c'],axis = 1)

In [ ]:
# training the model & prediction
model_logit = LogisticRegression(random_state=1234, max_iter = 1000, solver='liblinear') # old version(<0.22) default solver 로 통일
fit_logit = model_logit.fit(X_train_reduced, y_train)
y_pred_logit = fit_logit.predict(X_test_reduced)

## Model2 : 신경망
* hidden_layer_sizes=(3,4,5),random_state=1234, max_iter = 1000

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

In [ ]:
# data scaling 
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# training the model & prediction
model_nn = MLPClassifier(hidden_layer_sizes=(3,4,5),random_state=1234, max_iter = 1000)
fit_nn = model_nn.fit(X_train_scaled, y_train)
y_pred_nn =fit_nn.predict(X_test_scaled)

## Model3 : 의사결정나무

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# training the model & prediction
model_tree = DecisionTreeClassifier(random_state = 1234,  max_depth=5)
fit_tree = model_tree.fit(X_train,y_train)
y_pred_tree = fit_tree.predict(X_test)

## Model 4: 랜덤포레스트

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model_rf = RandomForestClassifier(random_state=1234)
fit_rf=model_rf.fit(X_train,y_train)
y_pred_rf = fit_rf.predict(X_test)

## Model 5: 서포트 벡터 머신 (SVM)
* linear kernel 이용

In [ ]:
from sklearn import svm

In [ ]:
# linear kernel 이용 
model_svm = svm.SVC(kernel='linear', random_state=1234)
fit_svm= model_svm.fit(X_train,y_train)
y_pred_svm = fit_svm.predict(X_test)

###### 모형비교
* 신경망 모델이 AUC = 0.86으로 가장 좋은 성능을 보인다.

In [ ]:
from sklearn.metrics import plot_roc_curve

roc_logit = plot_roc_curve(fit_logit, X_test_reduced, y_test)
roc_nn = plot_roc_curve(fit_nn, X_test_scaled, y_test, ax = roc_logit.ax_)
roc_tree = plot_roc_curve(fit_tree, X_test, y_test, ax = roc_logit.ax_)
roc_rf = plot_roc_curve(fit_rf, X_test, y_test, ax = roc_logit.ax_)
roc_svm = plot_roc_curve(fit_svm, X_test, y_test, ax = roc_logit.ax_)
roc_rf.figure_.suptitle("ROC curve comparison");